In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import precision_score,f1_score
import numpy as np

In [19]:
df=pd.read_excel('dataset_telegram.xlsx')
df.head(5)

,DATE,HEADLINE,TICKER,COMPANY NAME,SECTOR,SUBSECTOR,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE,negative_score,positive_score,neutral_score,compound_score,High,Low,Open,close,volume,adj close,avg High,avg low,avg open,avg close,avg volume,avg adj close,trend
0,2022-02-15,Religare Ent Says\nâ–¶ï¸Religare Finvest vict...,RELIGARE,Religare Enterprises Limited,Finance - Investments,Finance - Investments,EQ,2007-11-21,10,1,INE621H01010,10,0.226,0.000,0.774,-0.8779,125.000000,125.000000,120.750000,122.050003,122.050003,910705,159.109998,159.779999,152.860001,156.130000,156.130000,686379.6,bearish
1,2022-02-15,Ashok Leyland targets 40% ICV sales from CNG v...,ASHOKLEY,Ashok Leyland Limited,Auto - LCVs & HCVs,Auto - LCVs & HCVs,EQ,1995-05-25,1,1,INE208A01029,1,0.000,0.000,1.000,0.0000,125.000000,129.250000,127.199997,128.500000,127.602028,15896279,159.329999,161.720001,156.199997,158.689999,157.581058,16145476.0,bearish
2,2022-02-15,Cipla promoters plan to sell 2.5% of stock,CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,1995-02-08,2,1,INE059A01026,2,0.000,0.000,1.000,0.0000,916.549988,938.799988,920.099976,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4,bullish
3,2022-02-15,"2.05 cr shares of Cipla worth â‚¹1,880 cr trad...",CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,1995-02-08,2,1,INE059A01026,2,0.093,0.202,0.705,0.3400,916.549988,938.799988,920.099976,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4,bullish
4,2022-02-15,NMDC in focus\n\nIron Ore falls 10% on SGX as ...,NMDC,NMDC Limited,Mining & Minerals,Mining & Minerals,EQ,2008-03-03,1,1,INE584A01023,1,0.153,0.000,0.847,-0.4019,153.250000,153.250000,149.050003,149.300003,143.697586,17659797,186.379999,189.089999,182.490005,185.719998,178.750937,18851958.2,bullish


In [20]:
df.dropna(inplace=True)
df.shape

(5386, 29)

In [21]:
data = pd.concat([df.iloc[:1000, 1:5], df.iloc[:1000, -12:-2]], axis=1)
data['HEADLINE'].replace("[^a-zA-Z0-9\s]"," ",regex=True,inplace=True)

data.shape

(1000, 14)

# Combine Columns

In [22]:
corpus = []
for row in range(0,len(data.index)):
    corpus.append(' '.join(str(x) for x in data.iloc[row,0:14]))
corpus

['Religare Ent Says\n      Religare Finvest victim of fraud perpetuated by erstwhile mgmt\n      Have challenged process of RFL being classified as    fraud    account \n      RBI on Feb 11 said RFL restructuring can   t be implemented with REL being a promoter as lenders declared RFL a fraud exposure RELIGARE Religare Enterprises Limited Finance - Investments 125.0 120.75 122.0500031 122.0500031 910705 159.1099976 159.7799988 152.8600006 156.1300003 156.1300003',
 'Ashok Leyland targets 40  ICV sales from CNG variants ASHOKLEY Ashok Leyland Limited Auto - LCVs & HCVs 129.25 127.1999969 128.5 127.6020279 15896279 159.3299988 161.7200012 156.1999969 158.6899994 157.5810577',
 'Cipla promoters plan to sell 2 5  of stock CIPLA Cipla Limited Pharmaceuticals 938.7999878 920.0999756 921.8499756 917.3931274 8304774 1135.429993 1155.719995 1129.699988 1145.319995 1139.782751',
 '2 05 cr shares of Cipla worth    1 880 cr trade on NSE at average    915 sh\n\nAlert  Promoters likely to have sold 

In [23]:
y = pd.get_dummies(df['trend']).iloc[:1000,1].values
y

array([False, False,  True,  True,  True, False, False, False,  True,
       False,  True, False, False,  True, False,  True, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True, False,  True,  True,  True,  True,
       False, False,  True, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False, False,  True, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True, False,  True,
       False, False,

# One-hot Representation

In [24]:
vocab_size = 20000
onehot_reprs=[one_hot(words,vocab_size)for words in corpus]
print(onehot_reprs)

[[8973, 5375, 526, 8973, 342, 7780, 12864, 10652, 6923, 16644, 11006, 572, 10698, 7312, 7408, 12864, 5007, 2835, 2354, 19706, 10652, 15181, 2713, 19421, 7475, 6325, 13092, 5007, 5629, 12549, 650, 3804, 8114, 12531, 18580, 2835, 14379, 17274, 19706, 12032, 929, 5007, 14379, 10652, 3000, 8973, 8973, 10155, 884, 4431, 1505, 380, 42, 15774, 6084, 1279, 4289, 1279, 4289, 18619, 4377, 19952, 4377, 4533, 4015, 4773, 17957, 14320, 17957, 14320], [7628, 6268, 13284, 12417, 9272, 14176, 4864, 5610, 2773, 4172, 7628, 6268, 884, 7524, 19046, 11617, 7044, 7888, 8822, 6566, 17032, 13008, 8822, 18469, 19538, 4377, 6804, 9156, 11755, 17957, 6566, 10652, 18908, 6000, 19114], [11018, 445, 1846, 12443, 10872, 4514, 13008, 12864, 7034, 11018, 11018, 884, 9131, 8226, 16855, 7452, 13327, 5140, 12990, 11883, 17322, 18969, 1519, 15374, 12285, 2020, 8125, 14837, 5629, 9610, 14800, 29], [4514, 18778, 1865, 12207, 12864, 11018, 16413, 15924, 2848, 1865, 2139, 19421, 4865, 15870, 9519, 17166, 15092, 17170, 445, 1

In [25]:
sent_len=100
emb=pad_sequences(onehot_reprs,padding='pre',maxlen=sent_len)
print(emb)

[[    0     0     0 ... 14320 17957 14320]
 [    0     0     0 ... 18908  6000 19114]
 [    0     0     0 ...  9610 14800    29]
 ...
 [    0     0     0 ...  7347 17166  2272]
 [    0     0     0 ... 19425 12417 19425]
 [    0     0     0 ...  4635 10843 10060]]


# Building LSTM Model

In [26]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=150,input_length=sent_len))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 150)          3000000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                55040     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3055105 (11.65 MB)
Trainable params: 3055105 (11.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
X_train, X_test, y_train, y_test = train_test_split(emb, y, test_size=0.25, random_state=42)

In [29]:
model.fit(X_train, y_train, epochs=10, verbose=1, batch_size=64)

Epoch 1/10


12/12 [==============================] - 4s 112ms/step - loss: 0.6852 - accuracy: 0.5987
Epoch 2/10
12/12 [==============================] - 1s 115ms/step - loss: 0.6051 - accuracy: 0.6880
Epoch 3/10
12/12 [==============================] - 1s 108ms/step - loss: 0.5031 - accuracy: 0.9200
Epoch 4/10
12/12 [==============================] - 1s 105ms/step - loss: 0.2692 - accuracy: 0.9787
Epoch 5/10
12/12 [==============================] - 1s 107ms/step - loss: 0.1562 - accuracy: 0.9893
Epoch 6/10
12/12 [==============================] - 1s 102ms/step - loss: 0.0961 - accuracy: 0.9933
Epoch 7/10
12/12 [==============================] - 1s 107ms/step - loss: 0.0645 - accuracy: 0.9973
Epoch 8/10
12/12 [==============================] - 1s 114ms/step - loss: 0.0391 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 1s 112ms/step - loss: 0.0272 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 1s 109ms/step - loss: 0.0195 - accuracy: 1.0000


In [30]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("LSTM Model Accuracy:", accuracy)

8/8 [==============================] - 0s 8ms/step - loss: 0.7282 - accuracy: 0.7200
LSTM Model Accuracy: 0.7200000286102295


In [31]:
# Predictions
y_pred_prob = model.predict(X_test)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)  # Assuming threshold of 0.5

# Calculate F1 score and precision
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("F1 Score:", f1)
print("Precision:", precision)

8/8 [==============================] - 0s 8ms/step
F1 Score: 0.7083333333333334
Precision: 0.7456140350877193
